In [6]:
import numpy as np
from pyomo.environ import *

# Defining sets
T = 3  # Number of periods (t=1,2,3)
I = ['Site1', 'Site2']  # Set of sites
S = ['Scenario1', 'Scenario2', 'Scenario3']  # Scenarios

# Generating random demand scenarios for each site and scenario
np.random.seed(42)
demand = {
    s: {i: np.random.randint(50, 400) for i in I}
    for s in S
}

# Scenario probabilities (uniform)
prob = {s: 1/len(S) for s in S}

# Parameters
Q = 200  # Capacity per warehouse unit
f = 500  # Fixed cost per warehouse unit
r0 = 1.0  # Stage 1 capacity cost per unit
R0 = 300  # Stage 1 max capacity
rt = {t: 1.2 + 0.1*t for t in range(1, T+1)}  # Stage 2 capacity cost (increasing)
Rt = {t: 100 for t in range(1, T+1)}  # Stage 2 max capacity per period
ci = {'Site1': 2.0, 'Site2': 2.5}  # Cost to serve demand at site i
bt = {t: 5.0 + 0.5*t for t in range(1, T+1)}  # Unmet demand cost (increasing)
h = 0.1  # Holding cost per unit

###Pyomo Model
model = ConcreteModel()

# Sets
model.T = Set(initialize=range(1, T+1))  # Periods t >= 1
model.I = Set(initialize=I)  # Sites
model.S = Set(initialize=S)  # Scenarios


# Variables
# Stage 1 (t=0)
model.u0 = Var(within=NonNegativeReals)  # Initial capacity
model.y = Var(within=NonNegativeIntegers)  # Warehouse units rented

# Stage 2 (t >= 1, scenario-dependent)
model.u = Var(model.S, model.T, within=NonNegativeReals)  # Capacity added in period t
model.x = Var(model.S, model.I, model.T, within=NonNegativeReals)  # Relief to site i
model.w = Var(model.S, model.T, within=NonNegativeReals)  # Unmet demand
model.z = Var(model.S, model.T, within=NonNegativeReals)  # Inventory in depot


###Objective
def total_cost_rule(model):
    # Stage 1 costs: fixed + capacity
    stage1_cost = f * model.y + r0 * model.u0

    # Stage 2 expected costs: capacity + unmet + holding + relief
    stage2_cost = sum(
        prob[s] * (
            sum(rt[t] * model.u[s, t] for t in model.T) +  # Capacity expansion
            sum(bt[t] * model.w[s, t] for t in model.T) +  # Unmet demand
            sum(h * model.z[s, t] for t in model.T) +  # Holding
            sum(ci[i] * model.x[s, i, t] for i in model.I for t in model.T)  # Relief
        )
        for s in model.S
    )
    return stage1_cost + stage2_cost
model.total_cost = Objective(rule=total_cost_rule, sense=minimize)




###Constraints

# Stage 1: Capacity limits
def stage1_capacity_rule(model):
    return model.u0 <= Q * model.y
model.stage1_capacity = Constraint(rule=stage1_capacity_rule)

def stage1_max_rule(model):
    return model.u0 <= R0
model.stage1_max = Constraint(rule=stage1_max_rule)



# Stage 2: Resource acquisition limits
def stage2_capacity_rule(model, s, t):
    return model.u[s, t] <= Rt[t]
model.stage2_capacity = Constraint(model.S, model.T, rule=stage2_capacity_rule)



# Demand satisfaction (total relief <= total demand)
def demand_satisfaction_rule(model, s, i):
    return sum(model.x[s, i, t] for t in model.T) == demand[s][i]
model.demand_satisfaction = Constraint(model.S, model.I, rule=demand_satisfaction_rule)



# Available capacity (relief <= depot capacity)
def capacity_balance_rule(model, s, t):
    if t == 1:
        return sum(model.x[s, i, 1] for i in model.I) <= model.u0 + model.u[s, 1]
    else:
        return sum(model.x[s, i, t] for i in model.I) <= model.z[s, t-1] + model.u[s, t]
model.capacity_balance = Constraint(model.S, model.T, rule=capacity_balance_rule)


def capacity_balance_rule_2(model, s, t):
    if t == 1:
        return sum(model.x[s, i, 1] for i in model.I) <= model.u0 + model.u[s, 1]
    else:
      return sum(model.x[s, i, t] for i in model.I) <= model.u0 + sum(model.u[s, k] for k in range(1,t)) - sum(model.x[s, i, k] for i in model.I for k in range(1,t-1))
model.capacity_balance_2 = Constraint(model.S, model.T, rule=capacity_balance_rule_2)



# Inventory balance
def inventory_rule(model, s, t):
    if t == 1:
        return model.z[s, 1] == model.u0 + model.u[s, 1] - sum(model.x[s, i, 1] for i in model.I)
    else:
        return model.z[s, t] == model.z[s, t-1] + model.u[s, t] - sum(model.x[s, i, t] for i in model.I)
model.inventory = Constraint(model.S, model.T, rule=inventory_rule)



# Unmet demand
def unmet_demand_rule(model, s, t):
    return model.w[s, t] == sum(demand[s][i] for i in model.I) - sum(model.x[s, i, k] for i in model.I for k in range(1, t))
model.unmet_demand = Constraint(model.S, model.T, rule=unmet_demand_rule)




### Solve the Model
solver = SolverFactory('gurobi') 
results = solver.solve(model)

### Print Results
print("--- Stage 1 Decisions ---")
print(f"Warehouse units rented (y): {model.y.value}")
print(f"Initial capacity (u0): {model.u0.value}")

print("\n--- Stage 2 Decisions (Scenario1) ---")
for t in model.T:
    print(f"Period {t}:")
    print(f"  Capacity added (u): {model.u['Scenario1', t].value}")
    print(f"  Relief to sites (x): { {i: model.x['Scenario1', i, t].value for i in model.I} }")
    print(f"  Unmet demand (w): {model.w['Scenario1', t].value}")
    print(f"  Inventory (z): {model.z['Scenario1', t].value}")

print(f"\nTotal Expected Cost: {model.total_cost()}")

--- Stage 1 Decisions ---
Warehouse units rented (y): 2.0
Initial capacity (u0): 300.0

--- Stage 2 Decisions (Scenario1) ---
Period 1:
  Capacity added (u): 100.0
  Relief to sites (x): {'Site1': 152.0, 'Site2': 248.0}
  Unmet demand (w): 550.0
  Inventory (z): 0.0
Period 2:
  Capacity added (u): 100.0
  Relief to sites (x): {'Site1': 0.0, 'Site2': 100.0}
  Unmet demand (w): 150.0
  Inventory (z): 0.0
Period 3:
  Capacity added (u): 50.0
  Relief to sites (x): {'Site1': 0.0, 'Site2': 50.0}
  Unmet demand (w): 50.0
  Inventory (z): 0.0

Total Expected Cost: 5674.2
